# Diffusion Model
![](./images/gen-model-compare.webp)
![](./images/diffusion.webp)

## 前向过程

前向过程是往图片上加噪声的过程。给定真实图片$x_0 \sim q(x)$，diffusion前向过程通过$T$次累计对其添加高斯噪声：
$$ q(x_t|x_{t-1}) = \mathcal{N}(x_t; \sqrt{1-\beta_t}x_{t-1}, \beta_t I) $$
$\beta \in (0, 1)$是高斯分布方差的超参数，实际中是随着$t$增大而递增。当$T \to \infty$时，$x_T$是完全的高斯噪声。

![](./images/02.png)

### 特性1 重参数（reparameterization trick）

如果我们要从某个分布中随机采样(高斯分布)一个样本，这个过程是无法反传梯度的,需要通过重参数技巧来使得它可微。最通常的做法是把随机性通过一个独立的随机变量$\epsilon$引导过去。比如，如果要从高斯分布$z \sim \mathcal{N}(z; \mu, \sigma^2)$中采样一个$z$，可以写成:

$$ z = \mu_\theta + \sigma_\theta \odot \epsilon, \epsilon \sim \mathcal{N}(0, 1) $$

上式的$z$依旧是有随机性的， 且满足均值为$\mu_\theta$方差为$\sigma^2_\theta$的高斯分布。这里的$\mu_\theta$，$\sigma_\theta$可以是由参数$\theta$的神经网络推断得到的。整个“采样”过程依旧梯度可导，随机性被转嫁到了$\epsilon$上。

### 特性2：任意时刻的$x_t$可以由$x_0$和$\beta$表示
令$\alpha_t = 1 - \beta_t$，且$\bar \alpha_t = \prod_{i=1}^t \alpha_i$，展开$x_t$的表达式：

$$ x_t = \sqrt{\alpha_t}x_{t-1} + \sqrt{1-\alpha_t}z_t, z_t \sim \mathcal{N}(0, I) $$
$$ = \sqrt{\alpha_t}(\sqrt{\alpha_{t-1}}x_{t-2} + \sqrt{1-\alpha_{t-1}}z_{t-1}) + \sqrt{1-\alpha_t}z_t $$
$$ = \sqrt{\bar \alpha_t}x_0 + \sqrt{1 - \bar \alpha_t}\bar z $$

由于高斯分布可加性，即 $\mathcal{N}(0, (\sigma_1^2)I) + \mathcal{N}(0, (\sigma_2^2)I) \sim \mathcal{N}(0, (\sigma_1^2 + \sigma_2^2)I)$，所以:
$$ \sqrt{\alpha_t(1-\alpha_{t-1})}z_{t-1} + \sqrt{1-\alpha_t}z_t \sim \mathcal{N}(0, [\alpha_t(1-\alpha_{t-1}) + (1-\alpha_t)]I) = \mathcal{N}(0, (1-\alpha_t\alpha_{t-1})I) $$

系数$\sqrt{1-\beta}$可以保证，当$T \to \infty$时，$x_T \sim \mathcal{N}(0,1)$，即保证$x_T$最后收敛到方差为1的标准高斯分布，切让推导更为简洁。

## 逆向过程

如果说前向过程(forward)是加噪的过程，那么逆向过程(reverse)就是diffusion的去噪推断过程。使用深度学习模型（参数为$\theta$，目前主流是U-Net+attention的结构）去预测这样的一个逆向的分布$p_theta$（类似VAE）:
$$ p_\theta(X_{0:T}) = p(x_T) \prod_{t=1}^T p_\theta(x_{t-1}|x_t) $$
$$ p_\theta(x_{t-1}|x_t) = \mathcal{N}(x_{t-1}; \mu_\theta(x_t, t), \Sigma_\theta(x_t, t)) $$

结合贝叶斯公式：
$$ q(x_{t-1}|x_t,x_0) = \mathcal{N}(x_{t-1}; \tilde\mu(x_t, x_0), \tilde\beta_t I) $$
$$ q(x_{t-1}|x_t,x_0) = q(x_t|x_{t-1},x_0)\frac{q(x_{t-1}|x_0)}{q(x_t|x_0)} $$
$$ = exp(-\frac{1}{2}((\frac{\alpha_t}{\beta_t}+\frac{1}{1-\bar\alpha_{t-1}})x_{t-1}^2 - (\frac{2\sqrt{\alpha_t}}{\beta_t}x_t + \frac{2\sqrt{\bar\alpha_{t-1}}}{1-\bar\alpha_{t-1}}x_0)x_{t-1} + C(x_t,x_0))) $$

将逆向过程全部变回了前向。根据一般的高斯概率密度函数的指数部分 $ exp(-\frac{(x-\mu)^2}{2\sigma^2}) = exp(-\frac{1}{2}(\frac{1}{\sigma^2}x^2-\frac{2\mu}{\sigma^2}x + \frac{\mu^2}{\sigma^2})) $，整理得到：
$$ \frac{1}{\sigma^2} = \frac{1}{\tilde\beta_t} = (\frac{\alpha_t}{\beta_t}+\frac{1}{1-\bar\alpha_{t-1}}); \quad \beta_t=\frac{1-\bar\alpha_{t-1}}{1-\bar\alpha_{t}} \cdot \beta_t$$
$$ \frac{2\mu}{\sigma^2}=\frac{2\tilde\mu(x_t, x_0)}{\tilde\beta_t} = (\frac{2\sqrt{\alpha_t}}{\beta_t}x_t + \frac{2\sqrt{\bar\alpha_{t-1}}}{1-\bar\alpha_{t-1}}x_0) $$
$$ \tilde\mu(x_t, x_0) = \frac{\sqrt{\alpha_t}(1-\bar\alpha_{t-1})}{1-\bar\alpha_{t}}x_t + \frac{\sqrt{\bar\alpha_{t-1}}\beta_t}{1-\bar\alpha_{t}}x_0 $$
已知 $x_0=\frac{1}{\sqrt{\bar\alpha_t}}(x_t-\sqrt{1-\bar\alpha_t}\bar z_t)$，代入可得：
$$ \tilde\mu_t = \frac{1}{\sqrt{\alpha_t}}(x_t- \frac{\beta_t}{\sqrt{1-\bar\alpha_t}}\bar z_t)$$ 
其中高斯分布$\bar z_t$为深度模型所预测的噪声（用于去噪），得到：
$$ \mu_\theta(x_t,t)=\frac{1}{\sqrt{\alpha_t}}(x_t- \frac{\beta_t}{\sqrt{1-\bar\alpha_t}}z_\theta(x_t,t))$$ 

这样一来,DDPM的每一步的推断可以总结为：

1. 每个时间步通过$x_t$和$t$来预测高斯噪声 $z_\theta(x_t,t)$，得到均值$\mu_\theta(x_t,t)$

2. 得到方差 $\Sigma_\theta(x_t, t)$，DDPM中使用untrained $\Sigma_\theta(x_t, t)=\tilde\beta_t$ ，且认为 $\tilde\beta_t=\beta_t$ 和 $\tilde\beta_t=\frac{1-\bar\alpha_{t-1}}{1-\bar\alpha_t}\cdot\beta_t$ 结果近似，在GLIDE中则是根据网络预测trainable方差 $\Sigma_\theta(x_t, t)$
 .

3. 计算得到$q(x_{t-1}|x_t)$，利用重参数得到 $x_{t-1}$

![](./images/diffusion-back.webp)

## 训练
[推导过程](https://zhuanlan.zhihu.com/p/525106459)

diffusion训练的核心就是取学习高斯噪声$\bar z_t$和$z_\theta$之间的MSE。DDPM使用的简化loss为
$$ L_t = \mathbb{E}_{x_0, \bar z_t}[ || \bar z_t -z_\theta(\sqrt{ \bar\alpha_t} x_0 + \sqrt{1-\bar\alpha_t}\bar z_t, t) ||^2] $$
    

## Stable Diffusion
![](./images/stable-diffusion-unet-steps.png)

### Speed up - Diffusion on Latent
![](./images/diffusion-on-latent.webp)
![](./images/unet-with-text.png)
![](./images/unet-detail.png)